In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd

import dl_helper as helper


## Load Regional Masks

In [2]:
# TODO: Define the path_to_pkls_folder (where you stored the mask pkls)
group = '1'
pkl_folder = f'group_{group}'

path_to_pkls_folder = os.path.join('..', 'masks', pkl_folder)   

In [3]:
# get a list of .pkl files in path_to_pkls_folder
mask_filenames = [f for f in os.listdir(path_to_pkls_folder) if f.startswith('masked') and f.endswith('.pkl')]
bounding_file = [f for f in os.listdir(path_to_pkls_folder) if f.startswith('bounding') and f.endswith('.pkl')]
print("Masks: \t\t", mask_filenames)
print("Bounding box:\t", bounding_file)

Masks: 		 ['masked_array_region_009.pkl', 'masked_array_region_008.pkl', 'masked_array_region_007.pkl', 'masked_array_region_010.pkl']
Bounding box:	 ['bounding_box.pkl']


In [4]:
masked_arrays = {file.split('.')[-2].split('_')[-1]: pickle.load(open(os.path.join(path_to_pkls_folder, file), 'rb')) for file in mask_filenames}
bounding_box = pickle.load(open(os.path.join(path_to_pkls_folder, bounding_file[0]), 'rb'))

In [6]:
lons, lats = helper.get_lons_lats(bounding_box)

In [5]:
destination_foldername = f'means_group_{group}'
destination_path = os.path.join('..', 'means', destination_foldername)
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

In [6]:

def get_latest_file_number(destination_path):
    files = os.listdir(destination_path)
    files = [f for f in files if f.endswith('.csv')]

    file_numbers = [int(re.search(r'(\d+)\D*$', f).group(1)) for f in files]
    
    return max(file_numbers, default=-1) + 1

In [7]:
get_latest_file_number(destination_path)

500

In [9]:
chla_upperbound = 100 # throw away values above this threshold

# start = 9000      # used to get the last 145 days
# step = 9645-9500  # used to get the last 145 days

start = get_latest_file_number(destination_path)
step = 500

for _ in range(4):
    try:
        helper.process_mask_index(start, step, masked_arrays, bounding_box, chla_upperbound, destination_path)
        break  # If the operation succeeds, break the loop
    except RuntimeError as e:
        print(f"Error: {e}")
        start = get_latest_file_number(destination_path)  # Update start
    else:
        print("Operation failed after 2 attempts. Quitting.")

Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1


Error:curl error: Stream error in the HTTP/2 framing layer
Error:DAP DATADDS packet is apparently too short


Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1


Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1


Error:curl error: Stream error in the HTTP/2 framing layer
Note:Caching=1


Error: NetCDF: DAP failure


Note:Caching=1
Note:Caching=1
Note:Caching=1
Note:Caching=1


In [ ]:
# Inspect the data in the directory ../means
def inspect_data(directory):
    directory = '../means/'
    df = pd.DataFrame()

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            temp_df = pd.read_csv(filepath)
            df = pd.concat([df, temp_df])

    df['mean'] = df['mean'].replace('--', np.nan)
    df['region'] = df['region'].astype('category')
    df['datetime'] = pd.to_datetime(df['time'], unit='D', origin='1970-01-01')
    df['year'] = df['datetime'].dt.year
    df['mean'] = df['mean'].astype(float)

    return df
   

# Print the combined DataFrame
# print(inspect_data('../means'))